# AKS Automatic + MCP Weather Server Deployment Walkthrough

This notebook orchestrates deployment of the AKS Automatic infrastructure (defined in [`main.bicep`](main.bicep)) and publishes the MCP Weather Server (located in [`mcp-weather-server/src/index.ts`](mcp-weather-server/src/index.ts)) to the cluster.

## Prerequisites
- Azure CLI authenticated: `az login`
- Docker daemon running, logged into your Azure Container Registry (ACR)
- Bicep CLI installed (comes with Azure CLI 2.50+)
- Kubernetes CLI (`kubectl`) installed
- Notebook kernel with network and shell access (e.g., Azure ML compute, local dev machine)

In [ ]:
import json
import pathlib
import subprocess
import tempfile

# --- Configuration ---
RESOURCE_GROUP = "rg-aks-automatic"
LOCATION = "eastus"
DEPLOYMENT_NAME = "aks-automatic"
TEMPLATE_FILE = "main.bicep"
PARAM_FILE = "main.bicepparam"
ACR_NAME = "<your-registry>"  # e.g. myregistry
AKS_CLUSTER_NAME_OVERRIDE = None  # Optional manual override if you set a different clusterName in parameters
IMAGE_NAME = f"{ACR_NAME}.azurecr.io/mcp-weather-server:latest"
PROJECT_ROOT = pathlib.Path.cwd()

print(f"Resource group: {RESOURCE_GROUP}")
print(f"Deployment name: {DEPLOYMENT_NAME}")
print(f"ACR: {ACR_NAME}")
print(f"Image: {IMAGE_NAME}")

In [ ]:
# Step 1: Ensure the resource group exists
subprocess.run(["az", "group", "create",
                "--name", RESOURCE_GROUP,
                "--location", LOCATION,
                "--output", "none"], check=True)
print("Resource group ready.")

In [ ]:
# Step 2: Deploy AKS Automatic using Bicep
deploy = subprocess.run(["az", "deployment", "group", "create",
                           "--name", DEPLOYMENT_NAME,
                           "--resource-group", RESOURCE_GROUP,
                           "--template-file", TEMPLATE_FILE,
                           "--parameters", f"@{PARAM_FILE}",
                           "--output", "json"],
                          check=True, capture_output=True, text=True)
deployment_result = json.loads(deploy.stdout)
deployment_outputs = deployment_result.get("properties", {}).get("outputs", {})
print(json.dumps(deployment_outputs, indent=2))

In [ ]:
# Step 3: Capture key outputs for later steps
cluster_name = AKS_CLUSTER_NAME_OVERRIDE or deployment_outputs.get("clusterName", {}).get("value")
workspace_id = deployment_outputs.get("workspaceId", {}).get("value")
print(f"Cluster name: {cluster_name}")
print(f"Log Analytics workspace: {workspace_id}")
if not cluster_name:
    raise ValueError("clusterName output missing; check your deployment parameters and outputs.")

In [ ]:
# Step 4: Attach ACR permissions to the AKS cluster
subprocess.run(["az", "aks", "update",
                "--resource-group", RESOURCE_GROUP,
                "--name", cluster_name,
                "--attach-acr", ACR_NAME], check=True)
print("AKS cluster now has pull permissions for the ACR.")

In [ ]:
# Step 5: Merge AKS credentials into local kubeconfig
subprocess.run(["az", "aks", "get-credentials",
                "--resource-group", RESOURCE_GROUP,
                "--name", cluster_name,
                "--overwrite-existing"], check=True)
print(f"Kubeconfig updated for {cluster_name}.")

In [ ]:
# Step 6: Build and push MCP Weather Server image
server_dir = PROJECT_ROOT / "mcp-weather-server"
subprocess.run(["npm", "install"], cwd=server_dir, check=True)
subprocess.run(["npm", "run", "build"], cwd=server_dir, check=True)
subprocess.run(["docker", "build", "-t", IMAGE_NAME, "."], cwd=server_dir, check=True)
subprocess.run(["az", "acr", "login", "--name", ACR_NAME], check=True)
subprocess.run(["docker", "push", IMAGE_NAME], check=True)
print("Container image pushed to ACR.")

In [ ]:
# Step 7: Deploy MCP Weather Server workload to AKS
manifest_path = PROJECT_ROOT / "k8s" / "mcp-weather.yaml"
manifest_text = manifest_path.read_text()
patched_manifest = manifest_text.replace("<your-registry>.azurecr.io/mcp-weather-server:latest", IMAGE_NAME)
with tempfile.NamedTemporaryFile('w', suffix='.yaml', delete=False) as temp_manifest:
    temp_manifest.write(patched_manifest)
    rendered_manifest_path = temp_manifest.name
subprocess.run(["kubectl", "apply", "-f", rendered_manifest_path], check=True)
print("MCP Weather Server manifest applied.")

In [ ]:
# Step 8: Verify rollout
subprocess.run(["kubectl", "get", "pods", "-l", "app=mcp-weather-server"], check=True)
subprocess.run(["kubectl", "logs", "deploy/mcp-weather-server"], check=False)
print("Review pod status and logs above.")

## Next steps
- Attach your MCP-compatible Agent to the running pod (sidecar or exec).
- Configure RBAC/networking as needed for production environments.
- Extend the notebook with teardown steps (`kubectl delete`, `az group delete`) when you're ready to clean up.